# Translate data

In [1]:
#Imports
import pandas as pd
from deep_translator import GoogleTranslator
from tqdm import tqdm
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')  # Download sentence tokenizer data
from nltk.tokenize import sent_tokenize



[nltk_data] Downloading package punkt to /home/david/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/david/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
df = pd.read_csv("data/speeches2014-2024/speeches_2014-2015.csv")
display(df.head())

,file_id,speaker_name,speaker_party,speech_text,jaar,date,kamer,category,title,document_number,url,meta_url,vergadernummer
0,h-tk-20142015-31-10,De voorzitter,NaN,"Zoals de leden weten, hebben zij twee minuten ...",2014-2015,2014-12-02,tk,handelingen,10 Integratieonderwerpen,NaN,https://zoek.officielebekendmakingen.nl/h-tk-2...,https://zoek.officielebekendmakingen.nl/h-tk-2...,"nr. 31, item 10"
1,h-tk-20142015-31-10,Mevrouw Karabulut,SP,Voorzitter. Integratie is weer helemaal hot. D...,2014-2015,2014-12-02,tk,handelingen,10 Integratieonderwerpen,NaN,https://zoek.officielebekendmakingen.nl/h-tk-2...,https://zoek.officielebekendmakingen.nl/h-tk-2...,"nr. 31, item 10"
2,h-tk-20142015-31-10,Mevrouw Karabulut,SP,"Gegeven de actualiteit, de discussies en het d...",2014-2015,2014-12-02,tk,handelingen,10 Integratieonderwerpen,NaN,https://zoek.officielebekendmakingen.nl/h-tk-2...,https://zoek.officielebekendmakingen.nl/h-tk-2...,"nr. 31, item 10"
3,h-tk-20142015-31-10,De voorzitter,NaN,Bij een VAO is het handig om binnen de orde va...,2014-2015,2014-12-02,tk,handelingen,10 Integratieonderwerpen,NaN,https://zoek.officielebekendmakingen.nl/h-tk-2...,https://zoek.officielebekendmakingen.nl/h-tk-2...,"nr. 31, item 10"
4,h-tk-20142015-31-10,Mevrouw Karabulut,SP,Dit past heel erg binnen de orde van een debat...,2014-2015,2014-12-02,tk,handelingen,10 Integratieonderwerpen,NaN,https://zoek.officielebekendmakingen.nl/h-tk-2...,https://zoek.officielebekendmakingen.nl/h-tk-2...,"nr. 31, item 10"


In [3]:
display(df["speech_text"][0])
txt = df["speech_text"][0]

'Zoals de leden weten, hebben zij twee minuten spreektijd. In die twee minuten moeten zij moties indienen en kunnen zij eventueel nog een opmerking maken. Als eerste is het woord aan mevrouw Karabulut van de SP.'

In [4]:
def translate(text):
    return GoogleTranslator(source='auto', target='en').translate(text)


In [19]:
display(translate(txt))

'As the members know, they have two minutes of speaking time. In those two minutes they must submit motions and they can possibly make a comment. First is the floor to Mrs. Karabulut of the SP.'

## Split sentence
The translation api have limitation for google translation the text chant exeede 5000 characters

In [5]:
def split_sentences(text, max_chunk_size=4000):

    if len(text) <= max_chunk_size:
        return [text]

    
    # Split text into sentences
    sentences = sent_tokenize(text)
    
    chunks = []
    current_chunk = []
    current_length = 0


    
    # Split into chunks while preserving sentence boundaries
    for sentence in sentences:
        sentence_length = len(sentence)
        
        if current_length + sentence_length > max_chunk_size:
            chunks.append(' '.join(current_chunk))
            current_chunk = [sentence]
            current_length = sentence_length
        else:
            current_chunk.append(sentence)
            current_length += sentence_length
            
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    return chunks
def split_translate(chunks):
    # Translate chunks sequentially
    translated_text = []
    for chunk in chunks:
        try:
            translated = translate(chunk)
            translated_text.append(translated)
        except Exception as e:
            print(f"Error translating chunk: {e}")
            translated_text.append("")  # Add empty string on failure
            
    return ' '.join(translated_text)

def safe_translate(text, max_text_lenght=4000):
    chunks = split_sentences(text, max_chunk_size=max_text_lenght)
    return split_translate(chunks)

In [47]:
long_txt = """Voorzitter. Wat het CDA betreft, bespreken wij vandaag drie onderwerpen: de stelselverantwoordelijkheid van deze minister voor de verschillende decentralisaties, de AIVD en de problematiek met de identiteitswissels in Oost-Europa. Over de decentralisaties heeft het CDA nog altijd heel grote zorgen. En laat me duidelijk zijn: die zorgen nemen eerder toe dan af. Wij horen de staatssecretaris van VWS iedere keer weer de bezweringsformule uitspreken dat het allemaal wel goed komt. Nog erger is het dat hij in de provincies en in de diverse gemeenten allerlei toezeggingen doet waar we geen verzilvering van zien. Dat is heel ernstig. Deze minister moet namelijk ook weten dat het piept en kraakt in Nederland bij die taakverschuiving naar de gemeenten. Ondanks de brief die wij hebben gekregen, zien wij de stelselverantwoordelijkheid van deze minister op dit moment geen goede vorm krijgen. Ik geef een paar voorbeelden. Kijk naar de jeugdzorg, waarover we hadden afgesproken dat daarvoor vóór 1 november contracten zouden zijn gesloten. Dat is dus niet gelukt. De deadline is verschoven. Dit is niet alleen maar een organisatorisch dingetje. Er zijn daadwerkelijk ouders — en die hebben zich ook tot mij gewend — die zeggen: mijn zorginstelling kan niet meer toezeggen dat ik die zorg straks krijg, want ze weet niet hoeveel geld ze krijgt en ze weet niet of ze nog zal worden gecontracteerd. In Eindhoven is er een Europese aanbesteding. Hoe bizar kan het worden? De gemeente zegt vervolgens dat ze het wel probeert goed te krijgen, maar uiteindelijk is er van alles aan de hand. Wij willen dat de minister acteert om deze zorgen weg te nemen. Er is die vergeten groep. Het gaat om meer dan 300 miljoen euro. De staatssecretaris van VWS zegt dat we dat wel in de middelen zullen oplossen, maar wij zien daar nog geen plannen voor. Die willen wij krijgen van deze minister. De staatssecretaris van VWS is naar Zeeland gegaan. Hij heeft daar gezegd dat hij ook wel ziet dat het volume waarmee wordt gerekend, eigenlijk te weinig behelst, dat er dus wat bij moet en dat dit zal worden goedgemaakt. Maar als mijn collega Mona Keijzer daar in een motie om vraagt — het gaat immers direct om het Gemeentefonds — dan is zoiets ineens een motie die eng is, en wordt deze door de staatssecretaris ontraden. Wij willen dat deze minister, die verantwoordelijk is voor het totale stelsel, acteert. Het gaat namelijk niet goed. Een ander punt, naast die decentralisaties, zijn de "congruente samenwerkingsverbanden", zoals de minister ze altijd terecht noemt. Wij hebben al eerder gezegd dat wij de andere samenwerkingsverbanden zien. De veiligheidsregio's zijn anders dan de arbeidsmarktregio's, en ze zijn weer anders dan de regio's voor passend onderwijs. Het kan soms nog, maar nu wordt in de jeugd-GGZ een nieuw samenwerkingsverband opgelegd, die niet past bij de samenwerking die is opgelegd ten aanzien van passend onderwijs. Wij spreken de minister aan op de congruentie in die samenwerkingsverbanden. Wat gaat hij daarmee doen? Wij zien dat de gemeenten op dit moment echt gebukt gaan onder het organiseren van die taken. Wij zien dat ze eigenlijk geen ruimte meer hebben om te kijken naar de andere taken die ze hebben, bijvoorbeeld op het gebied van de ruimtelijke ordening en de agenda daarvoor. We lezen dat de infrastructuur in de gemeenten een beetje in elkaar zakt en dat het onderhoud van wegen op allerlei plekken echt tekortschiet. Niemand heeft daar nog een focus op, niemand heeft daar meer geld voor. Wij vinden dat een zorgpunt en willen dat de minister daar, met de VNG, aandacht aan besteedt. Wij hebben ook een zorgpunt over de samenwerking binnen de steden; het speerpunt. De heer Van Oosten heeft daar al vragen over gesteld, en daar sluit ik mij bij aan. Hetzelfde geldt voor de BAG. Wij hebben in het algemeen overleg van de minister de toezegging gekregen dat er voor de kazernegemeenten een oplossing zou komen. Deze dreigt echter niet in de decembercirculaire te worden vormgegeven. Dat willen wij wél. Nu de grensgemeenten in Nederland. Ook al is het een kikkerlandje, Nederland heeft heel veel grensgemeenten. Met name daar zien we op dit moment grote problemen. In die grensgemeenten liggen eigenlijk kansen, die niet worden benut. Sterker nog: daar komen heel grote knelpunten. Er gaan steeds meer taken naar de gemeenten, ook met betrekking tot het arbeidsmarktbeleid en in de zorg. De grensgemeenten kunnen eigenlijk geen kant op. Ze kunnen dingen eigenlijk maar naar één kant organiseren, terwijl dit zou moeten kunnen in een kring eromheen. Wij willen dat de minister van Binnenlandse Zaken ook op dit dossier gaat acteren. Dan kom ik bij het Gemeentefonds. De minister heeft op enig moment gezegd dat wij eigenlijk met een schone lei moeten beginnen: dit is het geld waar gemeenten het straks mee moeten doen en vervolgens zal het allemaal wel goed komen. En als de gemeenten echt met de rug tegen de muur zouden komen te staan, zou de gemeentelijke belasting niet erg omhoog mogen gaan om dat op te lossen."""
display(len(long_txt))
chunks = split_sentences(long_txt, 4000)
len(chunks)

5040

2

In [28]:
msg = {
    "translator": "google translate",
    "original test": txt,
    "normal_translation": translate(txt),
    "safe_translate": safe_translate(txt, 50)
}
display(msg)


{'translator': 'google translate',
 'original test': 'Zoals de leden weten, hebben zij twee minuten spreektijd. In die twee minuten moeten zij moties indienen en kunnen zij eventueel nog een opmerking maken. Als eerste is het woord aan mevrouw Karabulut van de SP.',
 'normal_translation': 'As the members know, they have two minutes of speaking time. In those two minutes they must submit motions and they can possibly make a comment. First is the floor to Mrs. Karabulut of the SP.',
 'safe_translate': ' As the members know, they have two minutes of speaking time. In those two minutes they must submit motions and they can possibly make a comment. First is the floor to Mrs. Karabulut of the SP.'}

## Apply to the dataset
This is going to take a while

In [ ]:
tqdm.pandas()
df['speech_text_google_translate'] = df['speech_text'].progress_apply(safe_translate)

  0%|                                                                                                                                       | 0/71175 [00:00<?, ?it/s]

  0%|▏                                                                                                                          | 78/71175 [01:15<15:02:39,  1.31it/s]

In [52]:
#display(df.head())
df['speech_text'][5]

'U verwijs naar een debat dat vorige week is afgesloten met het indienen van moties.'